In [ ]:
!pip install -q yargy


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('test_data.csv')


In [ ]:
data['insight'] = ''
data

,dlg_id,line_n,role,text,insight
0,0,0,client,Алло,
1,0,1,manager,Алло здравствуйте,
2,0,2,client,Добрый день,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,
4,0,4,client,Ага,
...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,
477,5,140,client,Спасибо спасибо,
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,


In [ ]:
from yargy import Parser, rule, and_, not_
from yargy.interpretation import fact
from yargy.predicates import gram, is_capitalized
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline
from yargy.predicates import caseless, normalized, dictionary


list_of_text=[]
polite_dialog=[]
for i in range(max(data['dlg_id'])+1):
  polite_dialog.append([False,False])
  list_of_text.append(list(data[data['role']=='manager'][data['dlg_id']==i]['text']))

Для работы с диалогами я буду использовать yargy-парсер, который удобно использовать для извлечения сущностей по определенному правилу

# Приветствие

Для приветствия мы будем использовать morph_pipеline, в котором пропишем возможные синонимичные выражения.

In [ ]:
pipeline_hello = morph_pipeline([
        'Добрый день',
        'Привет',
        'Здравствуйте',
        "Приветствую",
        "Доброго времени суток",
        "Добрый вечер",
        "Доброе утро"
    ])

Hello = fact(
    'Hello',
    ['first']
)
gnc = gnc_relation()
HELLO_NAME = rule(
    pipeline_hello.interpretation(
        Hello.first
    ).match(gnc)

).interpretation(
    Hello
)

parser = Parser(HELLO_NAME)
no_hello=[]
for i in range(len(list_of_text)):
  flag=False
  for j in range(len(list_of_text[i])):
     temp=parser.find(list_of_text[i][j])
     
     if temp!=None:
       print(f"номер диалога:{i} и номер строчки:{j}")
       print(f"реплика:{(list_of_text[i][j])}")
       data.loc[(data.text==list_of_text[i][j]), 'insight']=" greeting"
       polite_dialog[i][0]=True
       flag=True
       break
  if not flag:
    print(f"В диалоге {i} менеджер не поздоровался")

номер диалога:0 и номер строчки:0
реплика:Алло здравствуйте
номер диалога:1 и номер строчки:0
реплика:Алло здравствуйте
номер диалога:2 и номер строчки:0
реплика:Алло здравствуйте
номер диалога:3 и номер строчки:0
реплика:Алло дмитрий добрый день
В диалоге 4 менеджер не поздоровался
В диалоге 5 менеджер не поздоровался


# Прощание

In [ ]:
pipeline_bye = morph_pipeline([
'До свидания',
'Пока',
'всего доброго',
'всего хорошего',

    ])

Bye = fact(
    'Bye',
    ['first']
)
gnc = gnc_relation()
BYE_NAME = rule(
    pipeline_bye.interpretation(
        Bye.first
    ).match(gnc)

).interpretation(
    Bye
)

parser = Parser(BYE_NAME)

for i in range(len(list_of_text)):
  for j in range(len(list_of_text[i])-1,-1,-1):
     temp=parser.find(list_of_text[i][j])
     flag=False
     if temp!=None:
       print(f"номер диалога:{i} и номер строчки:{j}")
       print(f"реплика:{(list_of_text[i][j])}")
       polite_dialog[i][1]=True
       data.loc[(data.text==list_of_text[i][j]), 'insight']+="farewell"
       flag=True
       break
  if not flag:
    print(f"В диалоге {i} менеджер не попрощался")

номер диалога:0 и номер строчки:15
реплика:Всего хорошего до свидания
номер диалога:1 и номер строчки:28
реплика:До свидания
номер диалога:2 и номер строчки:39
реплика:Все хорошо
номер диалога:3 и номер строчки:21
реплика:Угу все хорошо да понедельника тогда всего доброго
номер диалога:4 и номер строчки:13
реплика:Во вторник все ну с вами да тогда до вторника до свидания
номер диалога:5 и номер строчки:79
реплика:Ну до свидания хорошего вечера


Я считаю что в данном случае все хорошо можно считать как разновидность прощания всего хорошего

# проверка на требование к менеджеру "В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом"

In [ ]:
for i in range(len(polite_dialog)):
  if(polite_dialog[i][0] and polite_dialog[i][1]):
    print(f"В диалоге {i} менеджер поздоровался и попрощался")
  elif (polite_dialog[i][0] and not polite_dialog[i][1]):
    print(f"В диалоге {i} менеджер поздоровался,но не попрощался")
  elif(not polite_dialog[i][0] and polite_dialog[i][1]):
    print(f"В диалоге {i} менеджер не поздоровался, но попрощался")
  elif(not polite_dialog[i][0] and not polite_dialog[i][1]):
    print(f"В диалоге {i} менеджер не поздоровался и не попрощался")

В диалоге 0 менеджер поздоровался и попрощался
В диалоге 1 менеджер поздоровался и попрощался
В диалоге 2 менеджер поздоровался и попрощался
В диалоге 3 менеджер поздоровался и попрощался
В диалоге 4 менеджер не поздоровался, но попрощался
В диалоге 5 менеджер не поздоровался, но попрощался


# название компании

для компании будем искать слово компания, после которого идут несколько существительных

In [ ]:
Company = fact(
    'Company',
    ['position', 'name']
)
Name = fact(
    'Name',
    ['first']
)


POSITION = morph_pipeline([
    'Компания'
])
NAME= gram('NOUN')
COMPANY= rule(rule(
    POSITION).interpretation(Company.position),rule(
    NAME,
    NAME.optional().repeatable()).interpretation(Company.name)
).interpretation(Company)

parser = Parser(COMPANY)
for i in range(len(list_of_text)):
  for j in range(len(list_of_text[i])):
     temp=parser.find(list_of_text[i][j])
     flag=False
     if temp!=None:
       print(f"номер диалога:{i} и номер строчки:{j}")
       print(f"компания:{temp.fact.name}")
       data.loc[(data.text==list_of_text[i][j]),'insight']+=f" company_name={temp.fact.name}"
       flag=True
       break
  if not flag:
    print(f"В диалоге {i} не было названия компании")

номер диалога:0 и номер строчки:1
компания:диджитал бизнес
номер диалога:1 и номер строчки:1
компания:диджитал бизнес
номер диалога:2 и номер строчки:1
компания:диджитал бизнес
номер диалога:3 и номер строчки:1
компания:китобизнес
В диалоге 4 не было названия компании
В диалоге 5 не было названия компании


# имя менеджера

Для имени менеджера воспользуемся грамматикой имен и пайплайном с ключевыми словами которые идут перед представлением имени

In [ ]:
POSITION = morph_pipeline([
'Меня зовут',
"я",
"это",
"мое имя"
    ])

Person = fact(
    'Person',
    ['position', 'name']
)
Name = fact(
    'Name',
    ['first']
)
FIRST = and_(
    gram('Name'),
    not_(gram('Abbr')),
)

gnc = gnc_relation()
NAME = rule(
    FIRST.interpretation(
        Name.first
    ).match(gnc)

).interpretation(
    Name
)
PERSON = rule(rule(
    POSITION).interpretation(Person.position),
    NAME.interpretation(Person.name)
).interpretation(Person)

parser = Parser(PERSON)
for i in range(len(list_of_text)):
  for j in range(len(list_of_text[i])):
     temp=parser.find(list_of_text[i][j])
     flag=False
     if temp!=None:
       print(f"номер диалога:{i} и номер строчки:{j}")
       print(f"имя:{temp.fact.name.first}")
       data.loc[(data.text==list_of_text[i][j]), 'insight']+=f" manager_name={temp.fact.name.first}"   
       flag=True
       break
  if not flag:
    print(f"В диалоге {i} менеджер не представился")

номер диалога:0 и номер строчки:1
имя:ангелина
номер диалога:1 и номер строчки:1
имя:ангелина
номер диалога:2 и номер строчки:1
имя:ангелина
номер диалога:3 и номер строчки:1
имя:максим
В диалоге 4 менеджер не представился
номер диалога:5 и номер строчки:0
имя:анастасия


In [ ]:
data

,dlg_id,line_n,role,text,insight
0,0,0,client,Алло,
1,0,1,manager,Алло здравствуйте,greeting
2,0,2,client,Добрый день,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,company_name=диджитал бизнес manager_name=анг...
4,0,4,client,Ага,
...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,
477,5,140,client,Спасибо спасибо,
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,
